In [ ]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os
import numpy as np
from collections import ChainMap
import itertools
from collections import defaultdict
from operator import methodcaller
from itertools import chain
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [ ]:
pop_2010_2019 = pd.read_excel('../data/raw_data/FL/pop_est_2010-2019_fl.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

In [ ]:
pop_2020_2021 = pd.read_excel('../data/raw_data/FL/pop_est_2020-2021_fl.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

In [ ]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.head()

In [ ]:
pop_combined.tail(15)

In [ ]:
metros = pd.read_excel('../data/raw_data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_fl = metros_df[metros_df['State Name']=='Florida']
metros_fl['County'] = metros_fl['County'].str.strip()
metros_fl['County'] = metros_fl['County'].str.upper()
metros_fl.head()

### Extracting data for year 2013 and combining with pop along with Metropolitans and Micropolitans

In [ ]:
file_2013 ='../data/raw_data/FL/FL_2013.pdf'

In [ ]:
fl_df_crashes13 = pd.DataFrame()
pages = list(range(43,45))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_crashes13 = fl_df_crashes13.append(df)

In [ ]:
fl_df_crashes13 = fl_df_crashes13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_crashes13 = fl_df_crashes13.iloc[:-2 , :]
fl_df_crashes13 = fl_df_crashes13.iloc[2: , :]
fl_df_crashes13.columns = ['County','total_crashes']
fl_df_crashes13.head()

In [ ]:
fl_df_crashes13.tail(15)

In [ ]:
fl_df_fatalaties13 = pd.DataFrame()
pages = list(range(45,47))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_fatalaties13 = fl_df_fatalaties13.append(df)

In [ ]:
fl_df_fatalaties13 = fl_df_fatalaties13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[:-2 , :]
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[2: , :]
fl_df_fatalaties13.columns = ['County','fatalities']
fl_df_fatalaties13.head()

In [ ]:
fl_df_fatalaties13.tail(15)

In [ ]:
fl_df_injuries13 = pd.DataFrame()
pages = list(range(47,49))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_injuries13 = fl_df_injuries13.append(df)

In [ ]:
fl_df_injuries13 = fl_df_injuries13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_injuries13 = fl_df_injuries13.iloc[:-2 , :]
fl_df_injuries13 = fl_df_injuries13.iloc[2: , :]
fl_df_injuries13.columns = ['County','injuries']
fl_df_injuries13.head()

In [ ]:
fl_df_injuries13.tail(15)

In [ ]:
fl_df_2013 = pd.merge(pd.merge(fl_df_fatalaties13,fl_df_injuries13,on='County'),fl_df_crashes13,on='County')
fl_df_2013 = fl_df_2013.drop(labels=63, axis=0)
fl_df_2013['County'] = fl_df_2013['County'].str.strip()
fl_df_2013['County'] = fl_df_2013['County'].str.upper()
fl_df_2013.head(10)

In [ ]:
fl_df_2013.tail(10)

In [ ]:
df_combined_fl_13 = pd.merge(fl_df_2013,pop_combined[['County', 2013]],on="County",how='left')
df_combined_fl_2013 = pd.merge(df_combined_fl_13,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2013.rename(columns = {2013:'population'}, inplace = True)
df_combined_fl_2013.insert(0,'Year',2013)    
df_combined_fl_2013.head(10)

In [ ]:
df_combined_fl_2013.tail(10)

In [ ]:
df_combined_fl_2013.to_excel('../data/cleaned_data/2013_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2014 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [ ]:
file_2014_2016 ='../data/raw_data/FL/FL_2016.pdf'

In [ ]:
fl_df_2014_2016 = pd.DataFrame()
pages = list(range(35,42))
pdf = pdfplumber.open(file_2014_2016)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2014_2016 = fl_df_2014_2016.append(df)

In [ ]:
fl_df_2014_2016 = fl_df_2014_2016.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2014_2016.head()

In [ ]:
fl_df_2014_2016.tail()

In [ ]:
fl_df_melt_2016 = pd.melt(fl_df_2014_2016,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2016 = fl_df_melt_2016.drop(['variable'],axis=1)
fl_df_melt_2016 = fl_df_melt_2016[['value',9, 16,2]]
fl_df_melt_2016.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2016.head(10)

In [ ]:
fl_df_melt_2016['County'] = fl_df_melt_2016['County'].str.strip()
fl_df_melt_2016['County'] = fl_df_melt_2016['County'].str.upper()
df_combined_fl_16 = pd.merge(fl_df_melt_2016,pop_combined[['County', 2016]],on="County",how='left')
df_combined_fl_2016 = pd.merge(df_combined_fl_16,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2016.rename(columns = {2016:'population'}, inplace = True)
df_combined_fl_2016.insert(0,'Year',2016) 
df_combined_fl_2016.head()

In [ ]:
fl_df_melt_2015 = pd.melt(fl_df_2014_2016,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2015 = fl_df_melt_2015.drop(['variable'],axis=1)
fl_df_melt_2015 = fl_df_melt_2015[['value',10,17,3]]
fl_df_melt_2015.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2015.head(10)

In [ ]:
fl_df_melt_2015['County'] = fl_df_melt_2015['County'].str.strip()
fl_df_melt_2015['County'] = fl_df_melt_2015['County'].str.upper()
df_combined_fl_15 = pd.merge(fl_df_melt_2015,pop_combined[['County', 2015]],on="County",how='left')
df_combined_fl_2015 = pd.merge(df_combined_fl_15,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2015.rename(columns = {2015:'population'}, inplace = True)
df_combined_fl_2015.insert(0,'Year',2015) 
df_combined_fl_2015.head()

In [ ]:
fl_df_melt_2014 = pd.melt(fl_df_2014_2016,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2014 = fl_df_melt_2014.drop(['variable'],axis=1)
fl_df_melt_2014 = fl_df_melt_2014[['value',11,18,4]]
fl_df_melt_2014.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2014.head(10)

In [ ]:
fl_df_melt_2014['County'] = fl_df_melt_2014['County'].str.strip()
fl_df_melt_2014['County'] = fl_df_melt_2014['County'].str.upper()
df_combined_fl_14 = pd.merge(fl_df_melt_2014,pop_combined[['County', 2014]],on="County",how='left')
df_combined_fl_2014 = pd.merge(df_combined_fl_14,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2014.rename(columns = {2014:'population'}, inplace = True)
df_combined_fl_2014.insert(0,'Year',2014) 
df_combined_fl_2014.head()

In [ ]:
df_combined_fl_2014.to_excel('../data/cleaned_data/2014_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2015.to_excel('../data/cleaned_data/2015_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2016.to_excel('../data/cleaned_data/2016_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for year 2017

In [ ]:
file_2017 ='../data/raw_data/FL/FL_2017.pdf'

In [ ]:
my_dict_p35 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[35].extract_text()
txt = txt[203:]
txt = txt[:-113]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p35['County'] = county
my_dict_p35['fatalities'] = fatalaties
my_dict_p35['injuries'] = injuries
my_dict_p35['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_p35['County'].append('Clay')
my_dict_p35['fatalities'].append('21')
my_dict_p35['injuries'].append('1,804')
my_dict_p35['total_crashes'].append('2,961')
print(my_dict_p35)

In [ ]:
my_dict_p36 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[36].extract_text()
txt = txt[120:]
txt = txt[:-70]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p36['County'] = county
my_dict_p36['fatalities'] = fatalaties
my_dict_p36['injuries'] = injuries
my_dict_p36['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_p36['County'].append('Glades')
my_dict_p36['fatalities'].append('13')
my_dict_p36['injuries'].append('153')
my_dict_p36['total_crashes'].append('202')
print(my_dict_p36)

In [ ]:
my_dict_p37 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[37].extract_text()
txt = txt[149:]
txt = txt[:-26]
county = re.findall("(\w+\s?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p37['County'] = county
my_dict_p37['fatalities'] = fatalaties
my_dict_p37['injuries'] = injuries
my_dict_p37['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_p37['County'] = ['Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson']
print(my_dict_p37)

In [ ]:
my_dict_p38 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[38].extract_text()
txt = txt[68:]
txt = txt[:-161]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p38['County'] = county
my_dict_p38['fatalities'] = fatalaties
my_dict_p38['injuries'] = injuries
my_dict_p38['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_p38['County'].append('Martin')
my_dict_p38['fatalities'].append('25')
my_dict_p38['injuries'].append('2,087')
my_dict_p38['total_crashes'].append('3,466')
print(my_dict_p38)

In [ ]:
my_dict_p39 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[39].extract_text()
txt = txt[75:]
txt = txt[:-120]
county = re.findall("(\w+.?\s?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p39['County'] = county
my_dict_p39['fatalities'] = fatalaties
my_dict_p39['injuries'] = injuries
my_dict_p39['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_p39['County'] = ['Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas']
my_dict_p39['County'].append('Polk')
my_dict_p39['fatalities'].append('112')
my_dict_p39['injuries'].append('7,555')
my_dict_p39['total_crashes'].append('11,378')
print(my_dict_p39)

In [ ]:
my_dict_p40 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[40].extract_text()
txt = txt[120:]
txt = txt[:-78]
county = txt.strip()
county = re.findall("(\w+.?\s?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p40['County'] = county
my_dict_p40['fatalities'] = fatalaties
my_dict_p40['injuries'] = injuries
my_dict_p40['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)


In [ ]:
my_dict_p40['County'] = ['Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns', 'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union']
my_dict_p40['County'].append('Volusia')
my_dict_p40['fatalities'].append('130')
my_dict_p40['injuries'].append('6,527')
my_dict_p40['total_crashes'].append('9,601')
print(my_dict_p40)

In [ ]:
my_dict_p41 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[41].extract_text()
txt = txt[160:]
txt = txt[:-26]
county = re.findall("(\w+\-?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p41['County'] = county
my_dict_p41['fatalities'] = fatalaties
my_dict_p41['injuries'] = injuries
my_dict_p41['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

In [ ]:
my_dict_combined = defaultdict(list)
dict_items = map(methodcaller('items'), (my_dict_p35,my_dict_p36,my_dict_p37,my_dict_p38,my_dict_p39,my_dict_p40,my_dict_p41))
for k, v in chain.from_iterable(dict_items):
    my_dict_combined[k].extend(v)
print(my_dict_combined)

In [ ]:
fl_df_2017 = pd.DataFrame(my_dict_combined)
fl_df_2017.head()

In [ ]:
fl_df_2017.tail()

In [ ]:
fl_df_2017['County'] = fl_df_2017['County'].str.strip()
fl_df_2017['County'] = fl_df_2017['County'].str.upper()
df_combined_fl_17 = pd.merge(fl_df_2017,pop_combined[['County', 2017]],on="County",how='left')
df_combined_fl_2017 = pd.merge(df_combined_fl_17,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2017.rename(columns = {2017:'population'}, inplace = True)
df_combined_fl_2017.insert(0,'Year',2017)
df_combined_fl_2017.head(15)

In [ ]:
df_combined_fl_2017.to_excel('../data/cleaned_data/2017_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2018 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [ ]:
file_2018_2020 ='../data/raw_data/FL/FL_2020.pdf'

In [ ]:
fl_df_2018_2020 = pd.DataFrame()
pages = list(range(38,46))
pdf = pdfplumber.open(file_2018_2020)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2018_2020 = fl_df_2018_2020.append(df)

In [ ]:
fl_df_2018_2020 = fl_df_2018_2020.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2018_2020 = fl_df_2018_2020.iloc[:-1 , :]
fl_df_2018_2020.head()

In [ ]:
fl_df_2018_2020.tail()

In [ ]:
fl_df_melt_2020 = pd.melt(fl_df_2018_2020,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2020 = fl_df_melt_2020.drop(['variable'],axis=1)
fl_df_melt_2020 = fl_df_melt_2020[['value',9, 16,2]]
fl_df_melt_2020.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2020['County'] = fl_df_melt_2020['County'].str.strip()
fl_df_melt_2020['County'] = fl_df_melt_2020['County'].str.upper()
fl_df_melt_2020.head(10)

In [ ]:
df_combined_fl_20 = pd.merge(fl_df_melt_2020,pop_combined[['County', 2020]],on="County",how='left')
df_combined_fl_2020 = pd.merge(df_combined_fl_20,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2020.rename(columns = {2020:'population'}, inplace = True)
df_combined_fl_2020.insert(0,'Year',2020)
df_combined_fl_2020.head()

In [ ]:
df_combined_fl_2020.tail(15)

In [ ]:
fl_df_melt_2019 = pd.melt(fl_df_2018_2020,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2019 = fl_df_melt_2019.drop(['variable'],axis=1)
fl_df_melt_2019 = fl_df_melt_2019[['value',10,17,3]]
fl_df_melt_2019.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2019.head(10)

In [ ]:
fl_df_melt_2019['County'] = fl_df_melt_2019['County'].str.strip()
fl_df_melt_2019['County'] = fl_df_melt_2019['County'].str.upper()
df_combined_fl_19 = pd.merge(fl_df_melt_2019,pop_combined[['County', 2019]],on="County",how='left')
df_combined_fl_2019 = pd.merge(df_combined_fl_19,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2019.rename(columns = {2019:'population'}, inplace = True)
df_combined_fl_2019.insert(0,'Year',2019)
df_combined_fl_2019.head()

In [ ]:
fl_df_melt_2018 = pd.melt(fl_df_2018_2020,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2018 = fl_df_melt_2018.drop(['variable'],axis=1)
fl_df_melt_2018 = fl_df_melt_2018[['value',11,18,4]]
fl_df_melt_2018.columns = ['County','fatalities','injuries','total_crashes']
fl_df_melt_2018.head(10)

In [ ]:
fl_df_melt_2018['County'] = fl_df_melt_2018['County'].str.strip()
fl_df_melt_2018['County'] = fl_df_melt_2018['County'].str.upper()
df_combined_fl_18 = pd.merge(fl_df_melt_2018,pop_combined[['County', 2018]],on="County",how='left')
df_combined_fl_2018 = pd.merge(df_combined_fl_18,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2018.rename(columns = {2018:'population'}, inplace = True)
df_combined_fl_2018.insert(0,'Year',2018)
df_combined_fl_2018.head(10)

In [ ]:
df_combined_fl_2018.to_excel('../data/cleaned_data/2018_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2019.to_excel('../data/cleaned_data/2019_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2020.to_excel('../data/cleaned_data/2020_FL.xlsx', engine='xlsxwriter',index=False)

### Combining all the files into a master file

In [ ]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_FL.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_fl_master = pd.concat(df, axis = 0)
df_fl_master['injuries'] = df_fl_master['injuries'].str.replace(',', '').astype(float)
df_fl_master['total_crashes'] = df_fl_master['total_crashes'].str.replace(',', '').astype(float)

In [ ]:
df_fl_master.isna().sum()

In [ ]:
df_fl_master.insert(1,'State','Florida')
df_fl_master.head(10)

In [ ]:
df_fl_master.tail(10)

In [ ]:
df_fl_master.to_excel('../data/FL.xlsx', engine='xlsxwriter',index=False)